In [29]:
import pandas as pd #display
import io #parsing tabulated data, various types of io
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
from google.colab import files
uploaded = files.upload()

Saving modified_data.csv to modified_data (3).csv


In [30]:
data = pd.read_csv(io.BytesIO(uploaded['modified_data.csv']))

In [24]:
print(data)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
0        1          60       RL         65.0     8450   Pave      Reg   
1        2          20       RL         80.0     9600   Pave      Reg   
2        3          60       RL         68.0    11250   Pave      IR1   
3        4          70       RL         60.0     9550   Pave      IR1   
4        5          60       RL         84.0    14260   Pave      IR1   
...    ...         ...      ...          ...      ...    ...      ...   
1455  1456          60       RL         62.0     7917   Pave      Reg   
1456  1457          20       RL         85.0    13175   Pave      Reg   
1457  1458          70       RL         66.0     9042   Pave      Reg   
1458  1459          20       RL         68.0     9717   Pave      Reg   
1459  1460          20       RL         75.0     9937   Pave      Reg   

     LandContour Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0            Lvl    AllPub    Inside  ... 

In [32]:
data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,IR1,Lvl,AllPub,Inside,...,0,320,0,0,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,IR1,Lvl,AllPub,Corner,...,228,0,0,0,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,Reg,Lvl,AllPub,Inside,...,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,1,2008,WD,Normal,118000


In [33]:
data.shape

(1460, 77)

In [34]:
original_features = list(data.columns)
len(original_features)

77

In [45]:
features_missing_values = list(data.columns[data.isnull().any()])
len(features_missing_values)

12

In [37]:
len(data)

1460

In [38]:
print(data.head(10))

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
0   1          60       RL         65.0     8450   Pave      Reg         Lvl   
1   2          20       RL         80.0     9600   Pave      Reg         Lvl   
2   3          60       RL         68.0    11250   Pave      IR1         Lvl   
3   4          70       RL         60.0     9550   Pave      IR1         Lvl   
4   5          60       RL         84.0    14260   Pave      IR1         Lvl   
5   6          50       RL         85.0    14115   Pave      IR1         Lvl   
6   7          20       RL         75.0    10084   Pave      Reg         Lvl   
7   8          60       RL          NaN    10382   Pave      IR1         Lvl   
8   9          50       RM         51.0     6120   Pave      Reg         Lvl   
9  10         190       RL         50.0     7420   Pave      Reg         Lvl   

  Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch PoolArea  \
0    AllPub    Inside  ...             0    

In [39]:
bfill, mean_fill= [],[]

In [40]:
from pandas.core.arrays.sparse import dtype
for feature in features_missing_values:
  print(feature, '', data[feature].isnull().sum()/len(data)*100,
        '',data[feature].dtype)
  if data[feature].dtype =='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage  17.73972602739726  float64
MasVnrType  0.547945205479452  object
MasVnrArea  0.547945205479452  float64
BsmtQual  2.5342465753424657  object
BsmtCond  2.5342465753424657  object
BsmtExposure  2.6027397260273974  object
BsmtFinType1  2.5342465753424657  object
BsmtFinType2  2.6027397260273974  object
Electrical  0.0684931506849315  object
FireplaceQu  47.26027397260274  object
GarageType  5.5479452054794525  object
GarageYrBlt  5.5479452054794525  float64
GarageFinish  5.5479452054794525  object
GarageQual  5.5479452054794525  object
GarageCond  5.5479452054794525  object


In [41]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [43]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace= True)

In [46]:
for feature in bfill:
  data[feature].fillna(method='bfill', inplace = True)

In [47]:
data.columns[data.isnull().any()]

Index(['FireplaceQu'], dtype='object')

In [48]:
data['FireplaceQu'].isnull().sum()

2

In [49]:
data['FireplaceQu'].fillna(method='ffill',inplace = True)

In [51]:
from pandas.core.arrays import categorical #encoding of dataset
categorical_features = list(data.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [52]:
numerical = list(set(original_features)-set(categorical_features))
numerical

['MoSold',
 'GrLivArea',
 'BsmtFinSF2',
 'BsmtFullBath',
 'OpenPorchSF',
 '3SsnPorch',
 'MasVnrArea',
 'GarageCars',
 'Fireplaces',
 'YearBuilt',
 'BsmtUnfSF',
 'OverallQual',
 'HalfBath',
 'MSSubClass',
 'MiscVal',
 'KitchenAbvGr',
 '2ndFlrSF',
 'GarageArea',
 'LotFrontage',
 'TotalBsmtSF',
 'FullBath',
 'BsmtHalfBath',
 'Id',
 'BsmtFinSF1',
 'YearRemodAdd',
 'ScreenPorch',
 'TotRmsAbvGrd',
 'SalePrice',
 'YrSold',
 '1stFlrSF',
 'BedroomAbvGr',
 'GarageYrBlt',
 'LowQualFinSF',
 'EnclosedPorch',
 'OverallCond',
 'PoolArea',
 'WoodDeckSF',
 'LotArea']

In [53]:
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [54]:
nominal = ['MSZoning', 'LotShape', 'HouseStyle']
ordinal = list(set(categorical_features)-set(nominal))

In [55]:
target=['SalePrice']

In [56]:
df_nominal=pd.get_dummies(data[nominal])

In [57]:
for feature in ordinal:
  data[feature]= data[feature].astype('category').cat.codes
df_ordinal= data[ordinal]

In [58]:
data[numerical]

,MoSold,GrLivArea,BsmtFinSF2,BsmtFullBath,OpenPorchSF,3SsnPorch,MasVnrArea,GarageCars,Fireplaces,YearBuilt,...,YrSold,1stFlrSF,BedroomAbvGr,GarageYrBlt,LowQualFinSF,EnclosedPorch,OverallCond,PoolArea,WoodDeckSF,LotArea
0,2,1710,0,1,61,0,196.0,2,0,2003,...,2008,856,3,2003.0,0,0,5,0,0,8450
1,5,1262,0,0,0,0,0.0,2,1,1976,...,2007,1262,3,1976.0,0,0,8,0,298,9600
2,9,1786,0,1,42,0,162.0,2,1,2001,...,2008,920,3,2001.0,0,0,5,0,0,11250
3,2,1717,0,1,35,0,0.0,3,1,1915,...,2006,961,3,1998.0,0,272,5,0,0,9550
4,12,2198,0,1,84,0,350.0,3,1,2000,...,2008,1145,4,2000.0,0,0,5,0,192,14260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,8,1647,0,0,40,0,0.0,2,1,1999,...,2007,953,3,1999.0,0,0,5,0,0,7917
1456,2,2073,163,1,0,0,119.0,2,2,1978,...,2010,2073,3,1978.0,0,0,6,0,349,13175
1457,5,2340,0,0,60,0,0.0,1,2,1941,...,2010,1188,4,1941.0,0,0,9,0,0,9042
1458,4,1078,1029,1,0,0,0.0,1,0,1950,...,2010,1078,2,1950.0,0,112,6,0,366,9717


In [62]:
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis =1)

In [63]:
new_data.shape


(1460, 91)

In [64]:
new_data.drop('SalePrice', axis=1, inplace=True)

In [65]:
X=new_data.to_numpy()

In [66]:
X[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 2.000e+00,
       0.000e+00, 2.000e+00, 5.000e+00, 6.000e+00, 1.300e+01, 4.000e+00,
       5.000e+00, 4.000e+00, 4.000e+00, 3.000e+00, 3.000e+00, 2.000e+00,
       1.000e+00, 4.000e+00, 3.000e+00, 2.000e+00, 1.000e+00, 0.000e+00,
       1.000e+00, 1.000e+00, 4.000e+00, 2.000e+00, 1.000e+00, 1.200e+01,
       1.000e+00, 0.000e+00, 1.000e+00, 4.000e+00, 8.000e+00, 4.000e+00,
       2.000e+00, 1.000e+00, 2.000e+00, 2.000e+00, 0.000e+00, 2.000e+00,
       1.710e+03, 0.000e+00, 1.000e+00, 6.100e+01, 0.000e+00, 1.960e+02,
       2.000e+00, 0.000e+00, 2.003e+03, 1.500e+02, 7.000e+00, 1.000e+00,
       6.000e+01, 0.000e+00, 1.000e+00, 8.540e+02, 5.480e+02, 6.500e+01,
       8.560e+02, 2.000e+00, 0.000e+00, 1.000e+00, 7.060e+02, 2.003e+03,
       0.000e+00, 8.000e+00, 2.008e+03, 8.560e+02, 

In [68]:
X=StandardScaler().fit_transform(X)

In [69]:
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.70420284, -0.16998114, -0.08304548,  0.76051192, -0.34339089,
       -0.09839655, -0.99453548, -0.07422696, -0.08712888,  1.51026451,
       -0.16124951, -0.21585871, -0.03174026, -0.41169079, -0.40936914,
       -1.20621453,  0.25597761,  0.75162623,  0.60466978,  0.30941909,
        0.303692  ,  0.18797343,  0.64046232,  0.31466687, -0.77797579,
        0.06423821,  0.22403433,  0.28780036,  0.28974476, -0.49151573,
       -0.02618016, -1.23804229, -0.23012211,  0.36420746, -0.29055229,
       -0.12579688,  0.74309239, -0.12304604, -0.22571613, -0.71775144,
        1.11993319,  0.31386709,  0.2085023 ,  0.83559903,  0.26381257,
       -0.40779461, -0.03628912, -0.88465767, -1.5991111 ,  0.37033344,
       -0.28865283,  1.10781015,  0.21650316, -0.11633929,  0.51141841,
        0.31172464, -0.95122649,  1.05099379, -0.94459061,  0.65147924,
        1.22758538,  0.07337496, -0.08768781, -0.21145358,  1.16

In [70]:
y = data[target]

In [71]:
y=y.to_numpy()
y.shape

(1460, 1)

In [72]:
y
y= StandardScaler().fit_transform(y)

In [73]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [74]:
X.shape

(1460, 90)

In [75]:
pca=PCA(n_components=2) #feature extraction

In [76]:
p_components = pca.fit_transform(X)
p_components

array([[ 2.2121708 ,  1.49428361],
       [ 0.23438714, -2.36871972],
       [ 2.84799184,  1.06139228],
       ...,
       [ 1.35628452,  2.99607179],
       [-2.798609  , -3.29510342],
       [-0.68557161, -2.77258281]])

In [77]:
pca.explained_variance_ratio_

array([0.11917905, 0.05645903])

In [78]:
#new dataset for training
reg = LassoCV()

In [79]:
reg.fit(p_components, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV()

In [80]:
reg.coef_

array([0.25999548, 0.03912524])

In [84]:
from sklearn import  linear_model
reg = linear_model.Ridge(alpha =.5)
reg.fit([[0,0], [0,0], [1,1]], [0, .1, 1])

reg.coef_

array([0.34545455, 0.34545455])

In [85]:
reg.intercept_

0.13636363636363638